In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augmentation

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [2]:
batchsize=8

In [3]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\sivaprakash\drowsiness detection\prepared data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\sivaprakash\drowsiness detection\prepared data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 67091 images belonging to 3 classes.
Found 16770 images belonging to 3 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\sivaprakash\drowsiness detection\prepared data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 11275 images belonging to 3 classes.


In [5]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(4,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [11]:
checkpoint = ModelCheckpoint(r'C:\Users\sivaprakash\drowsiness detection\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3 )

callbacks=[checkpoint,earlystop,learning_rate]

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=18)

C:\Users\SIVAPR~1\AppData\Local\Temp/ipykernel_4964/3119574698.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/18
7874/7874 [==============================] - ETA: 0s - loss: 0.2094 - accuracy: 0.9179
Epoch 00001: val_loss improved from inf to 0.17145, saving model to C:\Users\sivaprakash\drowsiness detection\models\model.h5
7874/7874 [==============================] - 637s 80ms/step - loss: 0.2094 - accuracy: 0.9179 - val_loss: 0.1714 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 2/18
7874/7874 [==============================] - ETA: 0s - loss: 0.1803 - accuracy: 0.9302
Epoch 00002: val_loss did not improve from 0.17145
7874/7874 [==============================] - 564s 72ms/step - loss: 0.1803 - accuracy: 0.9302 - val_loss: 0.1950 - val_accuracy: 0.9204 - lr: 0.0010
Epoch 3/18
7874/7874 [==============================] - ETA: 0s - loss: 0.1718 - accuracy: 0.9342
Epoch 00003: val_loss did not improve from 0.17145
7874/7874 [==============================] - 566s 72ms/step - loss: 0.1718 - accuracy: 0.9342 - val_loss: 0.1972 - val_accuracy: 0.9198 - lr: 0.0010
Epoch 4/18
7874/7874 [=========

In [6]:

model = tf.keras.models.load_model(r"C:\Users\sivaprakash\drowsiness detection\models\model.h5")

In [3]:
# additional saving of model
#model.save(r'C:\Users\sivaprakash\drowsiness detection\models\model_manual.h5')

NameError: name 'model' is not defined

In [ ]:
# Model Evaluation

In [10]:
loss_tr,acc_tr  = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\SIVAPR~1\AppData\Local\Temp/ipykernel_21220/1929272923.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_tr,acc_tr  = model.evaluate_generator(train_data)


0.9358361959457397
0.16283290088176727


In [8]:
loss_vr, acc_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\SIVAPR~1\AppData\Local\Temp/ipykernel_21220/2909778475.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_vr, acc_vr = model.evaluate_generator(validation_data)


0.9316695332527161
0.17185050249099731


In [11]:
loss_test, acc_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

C:\Users\SIVAPR~1\AppData\Local\Temp/ipykernel_21220/2428393938.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_test, acc_test = model.evaluate_generator(test_data)


0.9358361959457397
0.16283290088176727


In [20]:
# model1 = tf.keras.models.load_model(r'C:\Users\sivaprakash\drowsiness detection\models\model_manual.h5')

In [23]:
# loss_vr, acc_vr = model1.evaluate_generator(validation_data)
# print(acc_vr)
# print(loss_vr)

C:\Users\SIVAPR~1\AppData\Local\Temp/ipykernel_4964/1829394340.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model1.evaluate_generator(validation_data)


0.1669837087392807
0.9342097043991089


In [24]:
# acc_test, loss_test = model1.evaluate_generator(test_data)
# print(acc_tr)
# print(loss_tr)

C:\Users\SIVAPR~1\AppData\Local\Temp/ipykernel_4964/3755250476.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model1.evaluate_generator(test_data)


0.1617029309272766
0.9359631538391113


In [25]:
pred = model.predict(test_data)

In [4]:
# Convert the model to tflite.
model_check = tf.keras.models.load_model(r"C:\Users\sivaprakash\drowsiness detection\models\model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model_check)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\SIVAPR~1\AppData\Local\Temp\tmpsx5c1x36\assets


In [5]:
#convert model to quantized tflite model
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.post_training_quantize=True
tflite_quantized_model=converter.convert()
open("quantized_model_check.tflite", "wb").write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\SIVAPR~1\AppData\Local\Temp\tmprujc8prj\assets


INFO:tensorflow:Assets written to: C:\Users\SIVAPR~1\AppData\Local\Temp\tmprujc8prj\assets


22354240